### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Import Packages

In [12]:
import ads
from ads.common.auth import default_signer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from xgboost import XGBClassifier
from datetime import datetime
import io
import matplotlib.pyplot as plt
import oci

ads.set_auth(auth='resource_principal')

# Data import

In [6]:
bucket_name = '<your_bucket_name>'
file_name_import = 'adult_income' # dataset was taken from https://www.openml.org/search?type=data&sort=version&status=any&order=asc&exact_name=adult
namespace = '<your_namespace>'
df = pd.read_csv(f"oci://{bucket_name}@{namespace}/{file_name_import}", storage_options=default_signer())

# Model Training

In [8]:
X = df.drop('class', axis=1)
X=pd.get_dummies(X)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3) # 

In [9]:
model = XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.01,eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, ...)

# Export Result

In [15]:
y_test_pred = model.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred)
cm_test_pct = cm_test / cm_test.sum(axis=1, keepdims=True) * 100
cm_test_pct

array([[9.99548370e+01, 4.51630386e-02],
       [7.87269682e+01, 2.12730318e+01]])

In [16]:
ConfusionMatrixDisplay(cm_test_pct, display_labels=['<=50K', '>50K']).plot(cmap='Blues', values_format=".1f")

object_name=datetime.now().strftime("%Y-%m-%d %H:%M:%S")+'confusion_matrix.png'
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)

signer = oci.auth.signers.get_resource_principals_signer()
object_storage = oci.object_storage.ObjectStorageClient({}, signer=signer)
namespace = object_storage.get_namespace().data

object_storage.put_object(
    namespace_name=namespace,
    bucket_name='filesdemo',
    object_name=object_name,
    put_object_body=buf
)

plt.close()